In [5]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import matplotlib.pyplot as plt
import collections
import re
import pprint as pp
import numpy as np
import multiprocessing as mp
from multiprocessing import Process

import math
import gzip
import pickle as pkl
import time

from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_recall_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler


import tensorflow as tf
#tf.compat.v1.enable_eager_execution()

import fonctions
import modeles

import random
import os
random.seed(0)

from os import listdir
from tqdm.notebook import tqdm
from os.path import isfile, join
from multiprocessing.pool import ThreadPool
from sklearn.utils import shuffle

In [45]:
def descriptive_stats_on_chunk(chunk):
    
    directory = '/home/maxime/Desktop/RecSys2020/data/batches/'
    
    imbalance = {'reply_timestamp':[]  ,'retweet_timestamp':[],
                 'retweet_with_comment_timestamp':[],'like_timestamp':[] }
    
    proportions = {'hashtags':[], 'present_links':[], 'present_domains':[]}
    
    for batch_file in tqdm(chunk):
        
        X, y = fonctions.get_batch(directory, batch_file, False)
        
        taille = len(y.index)
        
        imbalance['reply_timestamp'].append( y['reply_timestamp'].value_counts()[1] / taille )
        imbalance['retweet_timestamp'].append( y['retweet_timestamp'].value_counts()[1] / taille )
        imbalance['retweet_with_comment_timestamp'].append( y['retweet_with_comment_timestamp'].value_counts()[1] / taille )
        imbalance['like_timestamp'].append( y['like_timestamp'].value_counts()[1] / taille )
        
        proportions['hashtags'].append( len( X.hashtags[ X.hashtags > 0 ]) / taille )
        proportions['present_links'].append( len( X.present_links[ X.present_links > 0 ]) / taille ) 
        proportions['present_domains'].append( len( X.present_domains[ X.present_domains > 0 ]) / taille )

    return imbalance, proportions

In [56]:
batch_path='/home/maxime/Desktop/RecSys2020/data/batches'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 8)

imbalance, proportions = descriptive_stats_on_chunk(chunks[0])

In [58]:
targets = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
trends = ['hashtags','present_links','present_domains']

imbalance_mean = [ np.mean(imbalance[target]) for target in targets]
imbalance_sd = [ np.std(imbalance[target]) for target in targets]

proportions_mean = [ np.mean(proportions[trend]) for trend in trends]
proportions_sd = [ np.std(proportions[trend]) for trend in trends]

In [60]:
pp.pprint(imbalance_mean)
pp.pprint(imbalance_sd)
pp.pprint(proportions_mean)
pp.pprint(proportions_sd)

[0.02690271186440678,
 0.11227468926553673,
 0.007690225988700564,
 0.4382363276836158]
[0.0009542590571955029,
 0.0024189489802351797,
 0.0005286023800893022,
 0.0038099730955308123]
[0.19825265536723163, 0.1376571186440678, 0.1376571186440678]
[0.0024603468268410656, 0.0020447086603468958, 0.0020447086603468958]


In [ ]:
### calculer la presence moyenne, quotidienne et la variance 
### des utilisateurs non reguliers sur un chunk pour pouvoir mettre ces valeurs dans la base d'entrainement :

In [14]:
import metric_loading

user = metric_loading.get_global_user_presence(False)

agregating user presence
formating user presence


In [15]:
mean_presence = sum( [ v[0] for v in tqdm(user.values()) ] )/3753173
print(mean_presence)

mean_daily_presence = sum( [ v[1] for v in tqdm(user.values()) ] )/3753173
print(mean_daily_presence)

mean_std_presence = sum( [ v[2] for v in tqdm(user.values()) ] )/3753173
print(mean_std_presence)


1.4379483706186738



0.2055893584021628



0.4120845783282576


In [16]:
author = metric_loading.get_global_author_presence(False)

author presence aggregation
author presence formating


In [17]:
mean_presence = sum( [ v[0] for v in tqdm(author.values()) ] )/5808253
print(mean_presence)

mean_daily_presence = sum( [ v[1] for v in tqdm(author.values()) ] )/5808253
print(mean_daily_presence)

mean_std_presence = sum( [ v[2] for v in tqdm(author.values()) ] )/5808253
print(mean_std_presence)


1.2841775315228177



0.18363738667565704



0.3888680324798434


In [ ]:
from sklearn.utils import class_weight

